In [1]:
import json
import urllib
from urllib.parse import urlparse
import httplib2 as http
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
all_busstops = pd.read_csv('data/all_busstops.csv')['0'].values.tolist()

In [3]:
len(all_busstops)

5040

In [ ]:
# API Call for just 1 minute of data
# update frequency- 1 minutes

while True: 
    bus_arrival_full = pd.DataFrame(columns = ['Operator', 'ServiceNo', 'time_extracted', 'BusStopCode',
       'OriginCode_NextBus1', 'DestinationCode_NextBus1',
       'EstimatedArrival_NextBus1', 'Latitude_NextBus1', 'Longitude_NextBus1',
       'VisitNumber_NextBus1', 'Load_NextBus1', 'Feature_NextBus1',
       'Type_NextBus1' ])    
    
    if __name__=="__main__":
     #Authentication parameters
        headers = { 'AccountKey' : 'pNhcp/4TTU69vdvLbYsuuw==','accept' : 'application/json'} #this is by default
        #count = 0
         #API parameters
        for bus_stop in all_busstops:
#             count +=1
#             if count % 100 == 0:
#                 print(count)
            uri = 'http://datamall2.mytransport.sg/' #Resource URL
            path = '/ltaodataservice/BusArrivalv2?BusStopCode='
            serviceno = bus_stop
            #Build query string & specify type of API call

            target = urlparse(uri + path + serviceno)
            method = 'GET'
            body = ''

            #Get handle to http
            h = http.Http()
            #Obtain results
            response, content = h.request(target.geturl(),method,body,headers)

            #Parse JSON
            jsonObj = json.loads(content)

            # from JASON to pandas dataframe  
            bus_arrival_now = pd.DataFrame.from_dict (jsonObj['Services'])

            # get current time
            currentDT = datetime.datetime.now()
            currentDT.strftime("%Y-%m-%d %H:%M:%S")

            #data manupulation, combine all data to a single dataframe
            try: 
                operator = bus_arrival_now[['Operator','ServiceNo']]
                operator['time_extracted'] = currentDT.strftime("%Y-%m-%d %H:%M:%S")
                operator['BusStopCode'] = jsonObj['BusStopCode']

                NextBus1 = bus_arrival_now['NextBus'].apply(pd.Series).add_suffix('_NextBus1')
        #         NextBus2 = bus_arrival_now['NextBus2'].apply(pd.Series).add_suffix('_NextBus2')
        #         NextBus3 = bus_arrival_now['NextBus3'].apply(pd.Series).add_suffix('_NextBus3')

                # join all subtables together
        #         bus_arrival_partial = operator.join(NextBus1).join(NextBus2).join(NextBus3)
                bus_arrival_partial = operator.join(NextBus1)

                # append to full dataset
                bus_arrival_full = bus_arrival_full.append(bus_arrival_partial, ignore_index = True)

            except:
                pass

    filename = 'data/bus_arrival_' + currentDT.strftime("%m-%d_%H%M%S")+ '.csv'
    bus_arrival_full.to_csv(filename)
    print(currentDT.strftime("%m-%d_%H:%M:%S"), 'completed')
    time.sleep(120) # sleep for 2 minutes

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


02-23_23:57:23 completed
02-24_00:07:44 completed
02-24_00:17:47 completed
02-24_00:27:34 completed
